In [1]:
from __future__ import print_function, unicode_literals

import datetime
import json
import re
import time
import traceback

import botornot
import pandas as pd
%matplotlib inline
import requests
import seaborn as sns

import matplotlib.pyplot as plt

In [2]:
twitter_app_auth = {'consumer_key': 'mVJcyNN5qFyk7GdBJvENVc329',
                  'consumer_secret': 'DbXUXB7EN77HmzQahobHeTPSWgcNI9t8kKzDoe5twfTORnjXxf', 
                  'access_token': '707770026573561856-Vfqm0iXrsBvFL2prQMQ2Gzu9eawWYIj',
                  'access_token_secret':'HSMqS4YkDBcB6mm9JL3vd6VeKlkKmNdMxmJ0inJQwNhXS'}

for api_key in twitter_app_auth.values(): 
    assert not api_key.startswith('COPY_')

In [3]:
mashape_key = '7RTLrXbiE7mshwBPS1rjRbmtISIYp1P5V21jsn5j6SCln2a5Nd'

assert not mashape_key.startswith('COPY_')

In [4]:
hashtags = ['#chicago']
start_date = datetime.date(2015, 10, 1) # year, month, day
end_date   = datetime.date(2015, 10, 12)

for hashtag in hashtags:
    assert hashtag.startswith('#')

In [5]:
mashape_method_url = 'https://osome-public.p.mashape.com/user-post-count'
params = {
    'q': ','.join(hashtags),
    'start': start_date.isoformat(),
    'end': end_date.isoformat(),
}

headers = {
    'X-Mashape-Key': mashape_key,
}

mashape_response = requests.get(mashape_method_url, params=params, headers=headers)
mashape_content = mashape_response.json()

print(json.dumps(mashape_content, indent=3, sort_keys=True))
result_url = mashape_content['result_url']   

{
   "job_id": "38060427-602a-4924-9aa5-ebe0f2fb9e55",
   "result_url": "http://osome.iuni.iu.edu/moe/api/mashape/result/38060427-602a-4924-9aa5-ebe0f2fb9e55"
}


In [6]:
result_response = requests.get(result_url)
print(result_response.text[:255])

001Idea(3431334724)	1
001rossdale(3320422324)	1
003a04f8c2054b7(2920659436)	1
016maronve(3911013077)	1
01DANTANA(143528181)	1
01umentze(3910535061)	1
0204Tsa(1684793863)	1
050everything(3688863976)	1
06umbrekkeg(3910598777)	1
072lguidihean(3917410277)	1
0


In [21]:
def extract_records(result_response):
    result_pattern = r'(\w+)\((\d+)\)\s+(\d+)'
    for line in result_response.text.split('\n'):
        match = re.match(result_pattern, line)
        if match:
            yield match.groups()

table = pd.DataFrame.from_records(extract_records(result_response),
                               columns=['screen_name', 'user_id', 'tweets'],
                               #index=['user_id'],
                              )

# from_records doesn't do type inference, so the tweets column gets parsed as a string
table['tweets'] = table['tweets'].astype(int)
table = table.sort_values(by='tweets', ascending=False)

table.head(10)

,screen_name,user_id,tweets
3471,johnlundin,16548855,185
2290,TheEarthNetwork,1723563360,154
296,Bentler,22653760,121
733,EI_Climate,155978068,93
441,Cecalli_Helper,1460288238,92
2915,climatehawk1,350509998,84
2102,ScotClimate,620439080,74
237,AssaadRazzouk,401775393,60
497,ClayEngberg,1524952110,59
609,DBrockstedt,622957208,53


In [23]:
climate_change_id = table['user_id'].tolist()

In [26]:
# save!

import pickle
path = 'C:\\Users\\chenwang\\Documents\\courses\\FA16\\CS597\\twitterMining\\InitialSet\\Chicago\\repo\\'
with open(path + 'osome_climate_change_userid.pickle','wb+') as f:
    pickle.dump(climate_change_id,f)